# Imports & Settings


In [1]:
# Imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from deep_translator import GoogleTranslator
import re 
from math import isnan

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_seq_items', None)

In [2]:
# dtype={'type': str} prevents being confused with data type for large data sets
train = pd.read_csv('data/train.csv', index_col='id', dtype={'type': str})
test = pd.read_csv('data/test.csv', index_col='id', dtype={'type': str})
train_translated = pd.read_csv('data/train_translated.csv', dtype={'type': str})
test_translated = pd.read_csv('data/test_translated.csv', index_col='id', dtype={'type': str})
combined_data = pd.read_csv('data/combined_data.csv', index_col='id', dtype={'type': str})
combined_data_translated = pd.read_csv('data/combined_data_translated.csv', index_col='id', dtype={'type': str})
combined_data_fully_translated = pd.read_csv('data/combined_data_fully_translated.csv', index_col='id', dtype={'type': str})
prep = pd.read_csv('data/prep.csv', index_col='id', dtype={'type': str})

# Parameter & Unit & Value

In [3]:
data = prep.copy()

In [9]:
data.unit.value_counts()

cm              5200
lk              1301
leht             942
24 x 36 mm       808
60 x 60 mm       391
mm               386
g                231
90 x 120 mm       55
60 x 95 mm        39
tk                37
100 x 150 mm      12
autoripoogen       7
sajand             6
130 x 180 mm       6
45 x 60 mm         3
A6                 2
180 x 240 mm       2
dm3                1
60 x 90 mm         1
number             1
10 x 15 cm         1
Name: unit, dtype: int64